Вторая часть проекта. Несбалансированные данные.

In [0]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report

In [0]:
df = pd.read_csv('All-seasons.csv')

In [0]:
top_speakers = df.groupby(['Character']).size().loc[df.groupby(['Character']).size() > 800]
print (top_speakers.sort_values(ascending=False))

Character
Cartman         9774
Stan            7680
Kyle            7099
Butters         2602
Randy           2467
Mr. Garrison    1002
Chef             917
Kenny            881
Sharon           862
dtype: int64


In [0]:
new_df = pd.DataFrame(df.loc[df['Character'].isin(top_speakers.index.values)])

In [0]:
X = new_df.Line
y = new_df.Character

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Сначала смотрим на результаты логистической регрессии на изначальных неравномерных данных

In [0]:
from imblearn.metrics import classification_report_imbalanced
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression())
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report_imbalanced(y_test, y_pred))
# обратим внимание на низкий recall у миноритарных классов

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


                    pre       rec       spe        f1       geo       iba       sup

     Butters       0.53      0.23      0.98      0.32      0.47      0.21       664
     Cartman       0.42      0.72      0.61      0.54      0.67      0.45      2363
        Chef       0.79      0.24      1.00      0.37      0.49      0.22       240
       Kenny       0.75      0.06      1.00      0.11      0.25      0.05       248
        Kyle       0.38      0.31      0.86      0.34      0.52      0.25      1753
Mr. Garrison       0.71      0.13      1.00      0.22      0.36      0.12       244
       Randy       0.60      0.14      0.99      0.23      0.38      0.13       631
      Sharon       0.86      0.26      1.00      0.40      0.51      0.24       218
        Stan       0.38      0.43      0.78      0.40      0.58      0.32      1960

 avg / total       0.47      0.42      0.81      0.39      0.55      0.30      8321



Пробуем поиграться с метриками

In [0]:
# пробуем прибавить больший вес миноритарным классам
from sklearn.linear_model import LogisticRegression
pipe = make_pipeline(TfidfVectorizer(), LogisticRegression(class_weight={'Kenny':45, 'Sharon':45, 'Mr. Garrison': 45}))
pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

print(classification_report_imbalanced(y_test, y_pred))
# У Кенни, Шэрон и Мистера Гаррисона действительно немного увеличилась полнота

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


                    pre       rec       spe        f1       geo       iba       sup

     Butters       0.73      0.11      1.00      0.20      0.34      0.10       664
     Cartman       0.50      0.51      0.80      0.51      0.64      0.40      2363
        Chef       0.81      0.07      1.00      0.13      0.27      0.06       240
       Kenny       0.09      0.56      0.83      0.16      0.68      0.45       248
        Kyle       0.44      0.21      0.93      0.29      0.45      0.18      1753
Mr. Garrison       0.11      0.57      0.86      0.18      0.70      0.47       244
       Randy       0.82      0.07      1.00      0.13      0.27      0.07       631
      Sharon       0.13      0.56      0.90      0.20      0.71      0.48       218
        Stan       0.44      0.25      0.90      0.32      0.47      0.21      1960

 avg / total       0.49      0.31      0.89      0.32      0.50      0.26      8321



Применяем RandomUnderSampler (случайно удаляем примеры из мажоритарных классов)

In [0]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomUnderSampler(),
                         LogisticRegression())

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [0]:
print(classification_report_imbalanced(y_test, y_pred))

                    pre       rec       spe        f1       geo       iba       sup

     Butters       0.27      0.39      0.91      0.32      0.59      0.33       664
     Cartman       0.49      0.28      0.88      0.36      0.50      0.24      2363
        Chef       0.20      0.45      0.95      0.28      0.66      0.41       240
       Kenny       0.11      0.49      0.87      0.17      0.65      0.41       248
        Kyle       0.35      0.24      0.88      0.29      0.46      0.20      1753
Mr. Garrison       0.14      0.40      0.93      0.21      0.61      0.35       244
       Randy       0.24      0.30      0.92      0.27      0.53      0.26       631
      Sharon       0.19      0.43      0.95      0.27      0.64      0.39       218
        Stan       0.39      0.23      0.89      0.29      0.45      0.19      1960

 avg / total       0.36      0.29      0.89      0.30      0.51      0.24      8321



In [0]:
# пусть модель поугадывает
new_text = ["what's the meaning of life?', 'why are we here?"]
print ("Это сказал", pipe.predict(new_text))

Это сказал ['Chef']


In [0]:
# верно))

применяем RandomOverSampler (случайно дублируем примеры из миноритарных классов)

In [0]:
from imblearn.over_sampling import RandomOverSampler
pipe = make_pipeline_imb(TfidfVectorizer(),
                         RandomOverSampler(),
                         LogisticRegression())

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


                    pre       rec       spe        f1       geo       iba       sup

     Butters       0.35      0.45      0.93      0.39      0.65      0.40       664
     Cartman       0.56      0.42      0.87      0.48      0.60      0.35      2363
        Chef       0.22      0.47      0.95      0.30      0.67      0.43       240
       Kenny       0.12      0.46      0.90      0.20      0.64      0.40       248
        Kyle       0.40      0.29      0.88      0.33      0.50      0.24      1753
Mr. Garrison       0.21      0.43      0.95      0.28      0.64      0.38       244
       Randy       0.29      0.35      0.93      0.32      0.57      0.30       631
      Sharon       0.22      0.44      0.96      0.29      0.65      0.40       218
        Stan       0.44      0.29      0.89      0.35      0.51      0.24      1960

 avg / total       0.42      0.36      0.89      0.38      0.57      0.31      8321



In [0]:
new_text = ["ooooh, that's exciting, my mother was from Alabama."]
print ("Это сказал", pipe.predict(new_text))

Это сказал ['Mr. Garrison']


In [0]:
# тоже угадал!))

метод SMOTE основан на идее генерации некоторого количества искусственных примеров, которые были бы «похожи» на имеющиеся в меньшем классе, но при этом не дублировали их.

In [0]:
from imblearn.over_sampling import SMOTE
pipe = make_pipeline_imb(TfidfVectorizer(),
                         SMOTE(),
                         LogisticRegression())

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred))
# тоже классно повышает качество

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


                    pre       rec       spe        f1       geo       iba       sup

     Butters       0.32      0.42      0.92      0.37      0.62      0.37       664
     Cartman       0.56      0.43      0.87      0.49      0.61      0.36      2363
        Chef       0.22      0.45      0.95      0.30      0.65      0.41       240
       Kenny       0.13      0.44      0.91      0.20      0.63      0.38       248
        Kyle       0.39      0.28      0.89      0.33      0.50      0.23      1753
Mr. Garrison       0.20      0.43      0.95      0.27      0.64      0.38       244
       Randy       0.27      0.34      0.92      0.30      0.56      0.30       631
      Sharon       0.24      0.44      0.96      0.31      0.65      0.40       218
        Stan       0.40      0.27      0.88      0.33      0.49      0.22      1960

 avg / total       0.41      0.36      0.89      0.37      0.56      0.30      8321



In [0]:
new_text = ["whoa-ho-ho-ho! great shot, pip!"]
print ("Это сказал", pipe.predict(new_text))

Это сказал ['Chef']
